In [ ]:
import gradio as gr
from dotenv import load_dotenv
import urllib
import google.generativeai as genai

load_dotenv() # load "/.env" file.

genai.configure() # load GOOGLE_API_KEY from "/.env" file

demo = gr.Interface()